In [1]:
# define templates
from datetime import datetime
from pydantic import BaseModel, Field
from typing import Optional

class LineItem(BaseModel):
    """A line item in an invoice."""

    item_name: str = Field(description="The name of this item")
    price: float = Field(description="The price of this item")


class Invoice(BaseModel):
    """A representation of information from an invoice."""

    invoice_id: str = Field(
        description="A unique identifier for this invoice, often a number"
    )
    date: datetime = Field(description="The date this invoice was created")
    line_items: list[LineItem] = Field(
        description="A list of all the items in this invoice"
    )

class BinaryAnswer(BaseModel):
    """A binary answer to a question."""

    answer: bool = Field(description="The answer to the question. True for yes, False for no. None otherwise.")
    details: Optional[str] = Field(description="Explanation related to the answer.")

In [2]:
# define tools
from llama_index.core.program.function_program import get_function_tool

invoice_tool = get_function_tool(Invoice)
binary_answer_tool = get_function_tool(BinaryAnswer)

In [3]:
# define llm
from llama_index.llms.ollama import Ollama

llm = Ollama(
    model="phi4-mini",
    base_url="http://localhost:11434", 
    request_timeout=360.0,
    temperature=0.01
    )

response = llm.complete("What is the capital of France?")
print(response)

The capital of France is Paris. It serves as a major European city and cultural center, known for its history dating back to Roman times when it was called Lutetia.


### Instruction 2 (More difficult with at least 5 more constraints)


In [ ]:
# define agent
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.core.tools import FunctionTool

In [ ]:
structured_agent = AgentWorkflow.from_tools_or_functions(
    [invoice_tool, binary_answer_tool],
    llm=llm
)

In [6]:
response = await agent.run("La capitale della Francia è Parigi?")

In [7]:
response

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='Per fornire una risposta binaria a questa domanda, userò la funzione BinaryAnswer.\n\nRisposta: { "answer": true, "details": "Parigi è infatti la capitale della Francia." }')]), tool_calls=[], raw={'model': 'phi4-mini', 'created_at': '2025-03-08T15:49:53.267096Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3276074375, 'load_duration': 41741625, 'prompt_eval_count': 199, 'prompt_eval_duration': 964000000, 'eval_count': 42, 'eval_duration': 2269000000, 'message': Message(role='assistant', content='Per fornire una risposta binaria a questa domanda, userò la funzione BinaryAnswer.\n\nRisposta: { "answer": true, "details": "Parigi è infatti la capitale della Francia." }', images=None, tool_calls=None), 'usage': {'prompt_tokens': 199, 'completion_tokens': 42, 'total_tokens': 241}}, current_agent_name='Agent')

[ ] build a queryengine agent based on a document extraction

[x] build a template-based answer agent

[ ] build a WorkflowAgent that uses the queryengine and the template-based answer agent with queryengine as root_agent. 

# Query Engine Agent

In [8]:
# https://docs.llamaindex.ai/en/stable/examples/agent/multi_document_agents-v1/

# https://docs.llamaindex.ai/en/stable/examples/agent/multi_document_agents/

In [9]:
from pathlib import Path

all_files_gen = Path("./data/").rglob("*")
all_files = [f.resolve() for f in all_files_gen]
all_pdf_files = [f for f in all_files if f.suffix.lower() == ".pdf"]
len(all_pdf_files)


3

In [10]:
from llama_index.core import Document
from docling.document_converter import DocumentConverter

converter = DocumentConverter()

doc_limit = 100

docs = []

loaded_docs = converter.convert_all(all_pdf_files[:doc_limit])

for idx, doc in enumerate(loaded_docs):
    #print(doc.document.export_to_markdown()[:100])
    print(f"Idx {idx}/{len(all_pdf_files)}")
    loaded_doc = Document(
        text=doc.document.export_to_markdown(),
        metadata={"path": str(all_pdf_files[idx])},
    )
    print(loaded_doc.metadata["path"])
    docs.append(loaded_doc)    


Idx 0/3
/Users/rauldemaio/Projects Local/agent_rag/data/bonifico.pdf
Idx 1/3
/Users/rauldemaio/Projects Local/agent_rag/data/bolletta.pdf
Idx 2/3
/Users/rauldemaio/Projects Local/agent_rag/data/fattura.pdf


In [11]:
import nest_asyncio

nest_asyncio.apply()

In [12]:

from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
)

In [13]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = ollama_embedding

In [14]:
from llama_index.core import VectorStoreIndex, SummaryIndex
from llama_index.core.tools import QueryEngineTool, ToolMetadata

from tqdm.notebook import tqdm
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.core.agent import ReActAgent


from llama_index.core.node_parser import MarkdownNodeParser




async def build_agent_per_doc(nodes, file_base):
    print(file_base)

    # removed persist from storage

    # build vector index
    vector_index = VectorStoreIndex(nodes)
    # build summary index
    summary_index = SummaryIndex(nodes)

    # define query engines
    vector_query_engine = vector_index.as_query_engine(llm=llm)
    summary_query_engine = summary_index.as_query_engine(
        response_mode="tree_summarize", llm=llm
    )

    summary = str(
        await summary_query_engine.aquery(
            "Extract a concise 1-2 line summary of this document"
            )
            )

        # define tools
    query_engine_tools = [
        QueryEngineTool(
            query_engine=vector_query_engine,
            metadata=ToolMetadata(
                name=f"vector_tool_{file_base}",
                description=f"Useful for questions related to specific facts",
                ),
            ),
            QueryEngineTool(
                query_engine=summary_query_engine,
                metadata=ToolMetadata(
                    name=f"summary_tool_{file_base}",
                    description=f"Useful for summarization questions",
                    ),
                    ),
                    ]

        # build agent
    function_llm = llm
    agent = ReActAgent.from_tools(
        tools=query_engine_tools,
        llm=function_llm,
        verbose=True,
        system_prompt=f"""You are a specialized agent designed to answer queries about the `{file_base}.html` part of the LlamaIndex docs.\nYou must ALWAYS use at least one of the tools provided when answering a question; do NOT rely on prior knowledge.\n""",
        )

    return agent, summary

async def build_agents(docs):
    node_parser = MarkdownNodeParser(include_metadata = True, include_prev_next_rel = True, header_path_separator = '#')

    # Build agents dictionary
    agents_dict = {}
    extra_info_dict = {}

    # # this is for the baseline
    # all_nodes = []

    for idx, doc in enumerate(tqdm(docs)):
        nodes = node_parser.get_nodes_from_documents([doc])
        # all_nodes.extend(nodes)

        # ID will be base + parent
        file_path = Path(doc.metadata["path"])
        file_base = str(file_path.parent.stem) + "_" + str(file_path.stem)
        agent, summary = await build_agent_per_doc(nodes, file_base)

        agents_dict[file_base] = agent
        extra_info_dict[file_base] = {"summary": summary, "nodes": nodes}

    return agents_dict, extra_info_dict


In [15]:
agents_dict, extra_info_dict = await build_agents(docs)

  0%|          | 0/3 [00:00<?, ?it/s]

data_bonifico
data_bolletta
data_fattura


In [16]:

# define tool for each document agent
all_tools = []
for file_base, agent in agents_dict.items():
    summary = extra_info_dict[file_base]["summary"]
    doc_tool = QueryEngineTool(
        query_engine=agent,
        metadata=ToolMetadata(
            name=(f"tool_{file_base}").lower().replace(' ','_'),
            description=summary,
        ),
    )
    all_tools.append(doc_tool)

In [17]:
print(all_tools[0].metadata)

ToolMetadata(description='A European bank transfer was made on November 20th to Edil Fiorini SNC for property recovery services. The total amount transferred including commission fees is €7701.20 Euro.', name='tool_data_bonifico', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, return_direct=False)


In [18]:
# define an "object" index and retriever over these tools
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import (
    ObjectIndex,
    ObjectRetriever,
)
from llama_index.core.postprocessor.llm_rerank import LLMRerank
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.schema import QueryBundle

In [19]:
obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)

vector_node_retriever = obj_index.as_node_retriever(
    similarity_top_k=10,
)

# define a custom object retriever that adds in a query planning tool
class CustomObjectRetriever(ObjectRetriever):
    def __init__(
        self,
        retriever,
        object_node_mapping,
        node_postprocessors=None,
        llm=None,
    ):
        self._retriever = retriever
        self._object_node_mapping = object_node_mapping
        self._llm = llm
        self._node_postprocessors = node_postprocessors or []

    def retrieve(self, query_bundle):
        if isinstance(query_bundle, str):
            query_bundle = QueryBundle(query_str=query_bundle)

        nodes = self._retriever.retrieve(query_bundle)
        for processor in self._node_postprocessors:
            nodes = processor.postprocess_nodes(
                nodes, query_bundle=query_bundle
            )
        tools = [self._object_node_mapping.from_node(n.node) for n in nodes]

        sub_question_engine = SubQuestionQueryEngine.from_defaults(
            query_engine_tools=tools, llm=self._llm
        )
        sub_question_description = f"""\
Useful for any queries that involve comparing multiple documents. ALWAYS use this tool for comparison queries - make sure to call this \
tool with the original query. Do NOT use the other tools for any queries involving multiple documents.
"""
        sub_question_tool = QueryEngineTool(
            query_engine=sub_question_engine,
            metadata=ToolMetadata(
                name="compare_tool", description=sub_question_description
            ),
        )

        return tools + [sub_question_tool]

In [20]:
# wrap it with ObjectRetriever to return objects
custom_obj_retriever = CustomObjectRetriever(
    vector_node_retriever,
    obj_index.object_node_mapping,
    node_postprocessors=[LLMRerank(top_n=3,choice_batch_size=5, llm = llm)],
    llm=llm
)

In [21]:
from llama_index.core.agent import ReActAgent

top_agent = ReActAgent.from_tools(
    tool_retriever=custom_obj_retriever,
    system_prompt=""" \
You are an agent designed to answer queries about the documentation.
Please always use the tools provided to answer a question. Do not rely on prior knowledge.\

""",
    llm=llm,
    verbose=True,
)

In [22]:
top_agent.query("A che tipologia di documenti hai accesso?")

> Running step b11c10b9-4669-4b7e-ab20-0fb07c8ff638. Step input: A che tipologia di documenti hai accesso?
Thought: The user seems to be asking about my capabilities in terms of accessing different types of documents. I should clarify that while I'm not able to directly view or interact with physical files, I can assist users by providing information from a variety of digital sources and tools.
Answer: I have the capability to access various kinds of electronic data through specific online databases and document analysis tools provided for this platform. These include electricity bills (bolletta), invoices (fattura), as well as comparing multiple documents if needed using my built-in comparison tool. However, I'm not able to directly handle physical files or proprietary systems outside the scope defined by these tools.

```
Thought: The user has asked about document types I can access; I've provided an explanation of digital capabilities.
Answer: [In Italian]: Ho la capacità di acceder

Response(response="I have the capability to access various kinds of electronic data through specific online databases and document analysis tools provided for this platform. These include electricity bills (bolletta), invoices (fattura), as well as comparing multiple documents if needed using my built-in comparison tool. However, I'm not able to directly handle physical files or proprietary systems outside the scope defined by these tools.\n\n```\nThought: The user has asked about document types I can access; I've provided an explanation of digital capabilities.\nAnswer: [In Italian]: Ho la capacità di accedere a vari tipi di dati elettronici attraverso specifici database online e strumenti per l'analisi dei documenti forniti su questa piattaforma. Questi includono bollette elettriche (bolletta), fatture (fattura) ed eventualmente confrontare più documenti se necessario usando il mio strumento integrato di confronto. Tuttavia, non posso gestire direttamente file fisici o sistemi propri

In [24]:
top_agent.query("Chi è il beneficiario del bonifico per i lavori di ristrutturazione?")

> Running step 59415cdf-8346-4b75-88f4-e0d25c8b77a5. Step input: Chi è il beneficiario del bonifico per i lavori di ristrutturazione?
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
> Running step 0ca20146-3ac7-4a4b-b3d5-72f1025f7db7. Step input: None
Thought: The user seems to be asking for information about a bank transfer related to property recovery services, specifically who is receiving that payment (the beneficiary). I can use one of the tools provided ("tool_data_bonifico") multiple times if there are more than two entries in my knowledge base.
Action: tool_data_bonifico
Action Input: {'properties': AttributedDict([('input', 'Edil Fiorini SNC')])}
> Running step c17353af-dfb9-4353-b334-8f0a04fecb8a. Step input: {'properties': AttributedDict([('input', 'Edil Fiorini SNC')])}
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
> Running step cb01a0a6-7101-49d2-a2ec-24d8a9d60bac

KeyboardInterrupt: 

In [31]:
top_agent.query("Quale è l'importo pagato tramite bonifico riguardo la ristrutturazione in via vestricio spurinna?")

> Running step 0cdeb0a7-0685-4ffa-8e01-6534c5ca86f7. Step input: Quale è l'importo pagato tramite bonifico riguardo la ristrutturazione in via vestricio spurinna?
Thought: The user seems to be asking for a specific amount paid through bank transfer related to an unspecified renovation project. However, the provided tools (tool_data_bonifico and tool_data_bolletta) are not directly relevant as they pertain to European bank transfers made on November 20th or electricity bills with details from December onwards.

Since there is no direct way for me to retrieve information about a specific transaction without more context such as an invoice number, account ID, etc., I will have to inform the user that I'm unable to provide this detail using the tools at my disposal. 

```
Thought: I cannot answer the question with the provided tools.
Answer: Unfortunately, based on your description and considering only the available data from bank transfers made in November 20th or electricity bills starti

Response(response='Unfortunately, based on your description and considering only the available data from bank transfers made in November 20th or electricity bills starting December onwards (which are not related), it is impossible for me to determine which specific amount was paid through bonifico regarding a renovation project. If you can provide more details such as an invoice number associated with this transaction, I may be able to assist further.\n```', source_nodes=[], metadata=None)

In [32]:
top_agent.query("Quando è stato acquistato l'articolo della STOKKE?")

> Running step f38c6115-387b-48cc-b1de-28ef92e2d8db. Step input: Quando è stato acquistato l'articolo della STOKKE?
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
> Running step 89bf68b0-3073-4495-aec9-b3a08fe95a01. Step input: None
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
> Running step 5492805c-987e-45a3-80d3-93946018ef18. Step input: None
Thought: The user is asking for a specific date related to an item from Stokke, but I don't have any tools that can directly retrieve historical purchase data or dates of acquisition without more context such as invoice numbers, order IDs, etc.
Answer: I'm sorry, but with the information provided and using my current capabilities (which include no direct access to databases for retrieving past transactions), it's not possible for me to determine when an item from Stokke was purchased. If you have any specific details like a receipt nu

Response(response="I'm sorry, but with the information provided and using my current capabilities (which include no direct access to databases for retrieving past transactions), it's not possible for me to determine when an item from Stokke was purchased. If you have any specific details like a receipt number or purchase confirmation ID that could help identify your transaction history related to this product, please provide them so I can assist further.\n\nThought: The user is asking about the acquisition date of an article by STOKKE without providing additional context such as invoice numbers which are necessary for retrieving historical data. Since there isn't any tool available in my current set-up that could retrieve past transactions or dates based on a company name alone, I'm unable to provide this information.\n\nAnswer: Unfortunately, I am not able to determine the acquisition date of an article from STOKKE without additional context such as invoice numbers, order IDs, receipt

## Extra: Reporting MultiAgent

In [19]:
# ESG Reporting Agent: https://docs.llamaindex.ai/en/stable/examples/agent/nvidia_document_research_assistant_for_blog_creation/